# 0. Setup Paths

In [1]:
import os
import wget
import object_detection

on tf Model Zoo Website (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)
- right click on model to get url
- copy name part of url to use as name

In [2]:
CUSTOM_MODEL_NAME = 'my_centernet_hg104_1024_8'
PRETRAINED_MODEL_NAME = 'centernet_hg104_1024x1024_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations',CUSTOM_MODEL_NAME),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'IMAGE_PATH_TRAIN': os.path.join('Tensorflow', 'workspace','images','train'),
    'IMAGE_PATH_TEST': os.path.join('Tensorflow', 'workspace','images','test'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
# creating folders
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [6]:
print(paths['CHECKPOINT_PATH'])
print(paths['ANNOTATION_PATH'])

Tensorflow/workspace/models/my_centernet_hg104_1024_8
Tensorflow/workspace/annotations/my_centernet_hg104_1024_8


# 1. Load pre-trained Model

In [7]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.8.12: /home/josefine/.conda/envs/tfod_v2/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-04-21 16:38:21.290446: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-21 16:38:21.683422: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10357 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:15:00.0, compute capability: 6.1
/home/josefine/.conda/envs/tfod_v2/lib/python3.8/site-packages/object_detection/builders/model_builder.py:1100: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC met

In [8]:
#!pip list
!nvidia-smi

Tue Apr 12 14:42:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:15:00.0 Off |                  N/A |
|  0%   52C    P0    75W / 250W |    169MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!who

dritonbahtiri :1           2022-03-28 08:43 (:1)
josefine pts/7        2022-04-19 13:38 (85.148.49.130)


In [8]:
# download model from tf model zoo
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2022-04-12 14:10:33--  http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.179.208, 2a00:1450:400e:803::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.179.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1424430731 (1,3G) [application/x-tar]
Saving to: ‘centernet_hg104_1024x1024_coco17_tpu-32.tar.gz’

centernet_hg104_102  98%[==================> ]   1,31G  45,1MB/s    in 22s     


Cannot write to ‘centernet_hg104_1024x1024_coco17_tpu-32.tar.gz’ (No space left on device).
centernet_hg104_1024x1024_coco17_tpu-32/
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/checkpoint
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/ckpt-0.index
centernet_hg104_1024x1024_coco17

# 2. Create Label Map

In [8]:
labels = [{'name':'Bubble', 'id':1}]

# write into label map file
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records
- binary storage format
- convert annotations and images into format for training
- storing data as sequence of binary strings has impact on training time
- (less space on disk, copy faster, read efficiently)

In [9]:
# create TFRecord file
# --> train and test record file within annotations path
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_8/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_8/test.record


# 4. Copy Model Config to Training Folder
- pipeline.config defines architecture of model
- contains e.g. prebuilt data augmentation
- see pipeline file in pre-trained model folder --> preset configurations (e.g. data augmentation etc.)

In [10]:
# copy pipeline.config into my_model folder
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning
config file: change parameter of the model

In [11]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [12]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [13]:
config

{'model': center_net {
   num_classes: 90
   feature_extractor {
     type: "hourglass_104"
     channel_means: 104.01361846923828
     channel_means: 114.03422546386719
     channel_means: 119.91659545898438
     channel_stds: 73.60276794433594
     channel_stds: 69.89082336425781
     channel_stds: 70.91507720947266
     bgr_ordering: true
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 1024
       max_dimension: 1024
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 100
   }
 },
 'train_config': batch_s

In [52]:
#ckeys = config.keys()
#print(type(ckeys))
#print(list(ckeys)[0])

In [14]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

MODIFY FIRST LINE!
- pipeline_config.model.###
- insert model name (shown in first line of config file)

In [15]:
# modify pipeline.config for personal use case (num classes, set paths etc.)
pipeline_config.model.center_net.num_classes = len(labels)
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [16]:
print([os.path.join(paths['ANNOTATION_PATH'], 'test.record')])

['Tensorflow/workspace/annotations/my_centernet_hg104_1024_8/test.record']


In [17]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model
Before Training, my_model directory must be empty!

In [6]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [19]:
# test if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# check GPU use
# watch -n0.1 nvidia-smi

Num GPUs Available:  1


In [20]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=20000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [21]:
# to see training status, copy command to prompt and run from there
# (from tfodCourse directory and with activated environment)
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_8 --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_8/pipeline.config --num_train_steps=20000


- runs model_main_tf2.py
- arguments passed:
    - model directory
    - directory of pipeline.config
    - number of training steps

In [39]:
#!{command}

2021-11-24 11:57:14.384791: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-24 11:57:14.956688: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10265 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:15:00.0, compute capability: 6.1
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1124 11:57:15.128776 140224440755008 mirrored_strategy.py:369] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 5000
I1124 11:57:15.132200 140224440755008 config_util.py:552] Maybe overwriting train_steps

# 7. Evaluate the Model
Calculate performance metrics

In [7]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [8]:
# command can be run from prompt to see progress
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_8 --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_8/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_8


In [75]:
#!{command}

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.254
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.274
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.035
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.135
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.325
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.050
 A

W1103 16:03:43.788491 17756 model_lib_v2.py:1081] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1103 16:03:43.789491 17756 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1103 16:03:43.789491 17756 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1103 16:03:43.789491 17756 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1103 16:03:43.789491 17756 model_lib_v2.py:1099] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2021-11-03 16:03:43.795207: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To 

I1103 16:04:26.952852 17756 model_lib_v2.py:1007] Eval metrics at step 2000
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP: 0.092891
I1103 16:04:26.955851 17756 model_lib_v2.py:1010] 	+ DetectionBoxes_Precision/mAP: 0.092891
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP@.50IOU: 0.254012
I1103 16:04:26.959856 17756 model_lib_v2.py:1010] 	+ DetectionBoxes_Precision/mAP@.50IOU: 0.254012
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP@.75IOU: 0.061814
I1103 16:04:26.965859 17756 model_lib_v2.py:1010] 	+ DetectionBoxes_Precision/mAP@.75IOU: 0.061814
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP (small): 0.001661
I1103 16:04:26.968851 17756 model_lib_v2.py:1010] 	+ DetectionBoxes_Precision/mAP (small): 0.001661
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP (medium): 0.274420
I1103 16:04:26.971855 17756 model_lib_v2.py:1010] 	+ DetectionBoxes_Precision/mAP (medium): 0.274420
INFO:tensorflow:	+ DetectionBoxes_Precision/mAP (large): -1.000000
I1103 16:04:26.975855 17756 model_lib_v2.py:1

# See Evaluation graphically with Tensorboard
- Eval folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\eval --> AV, AR
- Train folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\train --> losscurve

in prompt: go to directory

run command: tensorboard --logdir=.
    
open localhost url

## Ideas to improve performance
- more imgs of low performing classes to training dataset
- train longer
- change architecture